In [ ]:
'''
This notebook uses a MOGen model for route prediction with or without target information.
- specify the maritime traffic network to predict routes on
- specify training data to train the prediction model
- specify test data to evaluate the prediction model
- specify MOGen prediction model parameters
The notebook will train the prediction model, evaluate it based on test data and visualize evaluation metrics.
Optionally, predictions can be visualized against the ground truth on a map.
'''

In [ ]:
import warnings
import pickle
import sys
import numpy as np
import time

warnings.filterwarnings('ignore')

# add paths for modules
sys.path.append('../visualization')
sys.path.append('../features')
sys.path.append('../datawrangling')

# import modules
import dataloader_paths, dataloader_geo
import visualize
import geometry_utils
from maritime_traffic_network import MaritimeTrafficNetwork
from MOGen_path_prediction import MOGenPathPrediction
import prediction_model_evaluation_metrics as metrics

In [ ]:
# Specify maritime traffic network
datasize = 'full'
location = 'tromso'
model_date = '202204'
DP_tol = 10
min_samples = 13

# Specify training data
train_dates = ['202204', '202205', '202207']
train_filter = None        #None, 'Last', 'Passasjer', 'Tank',  'Fisk', 'Unknown', 'Auxiliary'

# Specify test data
test_dates = ['202206']
test_filter = None         #None, 'Last', 'Passasjer', 'Tank',  'Fisk', 'Unknown', 'Auxiliary'
selection_start = 0        # for sampling
selection_end = -1         # for sampling
selection_step = 200        # for sampling

# Specify prediction model parameters
prediction_task = 'next_nodes'  # 'next_nodes' (without destination information) or 'path' (with destination information)
max_order = 3                   # Maximum order of the MOGen model
model_selection = True          # if True, optimal order is determined automatically by the model
training_mode = 'partial'       # 'partial' (trained on original paths) or 'full' (trained on original paths and all subpaths)
n_walks = 1000                  # number of random walks for sampling
n_start_nodes = 1               # number of observed nodes (defaults to 1)
n_steps = 10                    # prediction horizon (only needed for next_nodes prediction)
n_predictions = 1               # top n_predictions predictions will be output
eval_mode = 'path'              # evaluation mode: 'path' evaluates against the ground truth path, 
                                #                  'trajectory' against the ground truth trajectory

In [ ]:
# Load maritime traffic network from pickle
network_name = model_date+'_waypoints_DP'+str(DP_tol)+'_HDBSCAN'+str(min_samples)+'_'+location+'_'+datasize+'_UTM'
network_path = '../../models/networks/best_networks/' + network_name + '.obj'
fileObj = open(network_path, 'rb')
network = pickle.load(fileObj)
fileObj.close()
network.hyperparameters

In [ ]:
# Load training data from file
path_prefix = '../../data/paths/'
training_paths = dataloader_paths.load_path_training_data(path_prefix, network_name, train_dates, filter=train_filter, data_version='')

In [ ]:
# Train MOGen Model
model = MOGenPathPrediction()
print(f'Training MOGen Model on {len(training_paths)} paths...')
model.train(training_paths, max_order=max_order, model_selection=model_selection, training_mode=training_mode)
model.model.plot()

In [ ]:
# Load test data from file
traj_path_prefix = '../../data/processed/'
all_test_paths = dataloader_paths.load_path_test_data(path_prefix, network_name, test_dates, 
                                                      0, -1, 1, filter=test_filter, data_version='')
test_trajectories = dataloader_geo.load_trajectories(traj_path_prefix, location, network.crs, test_dates)

In [ ]:
# Sample test data
if prediction_task == 'next_nodes':
    # split test paths in subpaths
    sub_paths = dataloader_paths.split_path_data(all_test_paths, n_steps+n_start_nodes)
    if selection_end == -1:
        selection_end = len(sub_paths)
    selection = np.arange(selection_start, selection_end, selection_step)
    test_paths = sub_paths.iloc[selection]
else:
    if selection_end == -1:
        selection_end = len(all_test_paths)
    selection = np.arange(selection_start, selection_end, selection_step)
    test_paths = all_test_paths.iloc[selection]

In [ ]:
# Predict
start_time = time.time()

predictions = model.predict(prediction_task, test_paths, network.G, 
                            n_start_nodes=n_start_nodes, n_steps=n_steps, 
                            n_predictions=n_predictions, n_walks=n_walks)

end_time = time.time()  # end timer
print(f'Time elapsed: {(end_time-start_time)/60:.2f} minutes')
pps = len(test_paths)/(end_time-start_time)
print('Predictions per second: ', pps)

In [ ]:
# Evaluate
evaluation_results, fig = metrics.evaluate_given_predictions(prediction_task, predictions, test_trajectories, 
                                                             network, n_start_nodes=n_start_nodes, n_steps=n_steps, eval_mode=eval_mode)

In [ ]:
# plot predictions with the MOGen model
prediction_task = 'next_nodes'
example = test_paths.iloc[3]
mmsi = example.mmsi
start_node = [example.path[0]]
end_node = example.path[-1]
true_path = example.path
trajectory = test_trajectories.get_trajectory(mmsi)

if prediction_task == 'next_nodes':
    predictions = model.predict_next_nodes(start_node=start_node, G=network.G, n_predictions=3, n_steps=3, n_walks=200)
if prediction_task == 'path':
    predictions, flag = model.predict_path(start_node, end_node, network.G, n_predictions=5, n_walks=2000, verbose=True)

map = visualize.map_prediction_and_ground_truth(predictions, start_node, trajectory, true_path, network, 
                                                min_passages=1, opacity=0.2, location=location, end_node=end_node)
map.save('../../reports/maps/'+network_name+'_MOGen_predictions.html')
print(f'Map saved under {"../../reports/maps/"+network_name+"_MOGen_predictions.html"}')